In [24]:
#import the necessary libraries
import pandas as pd
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
import seaborn as sns
sns.set_style('darkgrid')

from statsmodels.formula.api import ols
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
import scipy.stats as stats
from sklearn.model_selection import train_test_split

# Import Data:

In [2]:
data_1 = pd.read_csv("analytic_data2019.csv", skiprows=0)

/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (0,1,2,5,6,7,8,9,10,11,12,13,14,15,16,17,18,19,20,21,22,23,24,25,26,27,28,29,30,31,32,33,34,35,36,37,38,39,40,41,42,43,44,45,46,47,48,49,50,51,52,53,54,55,56,57,58,59,60,61,62,63,64,65,66,67,68,69,70,71,72,73,74,75,76,77,78,79,80,81,82,83,84,85,86,87,88,89,90,91,92,93,94,95,96,97,98,99,100,101,102,103,104,105,106,107,108,109,110,111,112,113,114,115,116,117,118,119,120,121,122,123,124,125,126,127,128,129,130,131,132,133,134,135,136,137,138,139,140,141,142,143,144,145,146,147,148,149,150,151,152,153,154,155,156,157,158,159,160,161,162,163,164,165,166,167,168,169,170,171,172,173,174,175,176,177,178,179,180,181,182,183,184,185,186,187,188,189,190,191,192,193,194,195,196,197,198,199,200,201,202,203,204,205,206,207,208,209,210,211,212,213,214,215,216,217,218,219,220,221,222,223,224,225,226,227,228,229,230,231,232,233,234,235,236,237,238,239,240,241,242,243,244,245,246,247,248,249,250,251,2

In [3]:
data_1 = data_1.iloc[1:]

In [4]:
# cols1 = data_1.columns
# for col in cols1:
#     print(col)

In [5]:
xls = pd.ExcelFile('2019 County Health Rankings Data - v2.xls')
sheet0 = xls.parse(0,skiprows=1) 
sheet1 = xls.parse(1,skiprows=1) 
sheet2 = xls.parse(2,skiprows=1) 
sheet3 = xls.parse(3,skiprows=1) 
sheet4 = xls.parse(4,skiprows=1) 
sheet5 = xls.parse(5,skiprows=1) 
sheet6 = xls.parse(6,skiprows=1) 

Merge the table's into 1

In [6]:
combimed = pd.concat([data_1, sheet5, sheet3], axis=1, sort=False)
combimed.shape

(3195, 807)

In [7]:
cols1 = combimed.columns
cols1 = sorted(cols1)
for col in cols1:
    print(col)

# African American
# Alcohol-Impaired Driving Deaths
# American Indian/Alaskan Native
# Asian
# Associations
# Chlamydia Cases
# Deaths
# Deaths.1
# Deaths.2
# Dentists
# Driving Deaths
# Drug Overdose Deaths
# Firearm Fatalities
# Food Insecure
# HIV Cases
# Hispanic
# Homeowners
# Households
# Households with Severe Cost Burden
# Injury Deaths
# Limited Access
# Mental Health Providers
# Motor Vehicle Deaths
# Native Hawaiian/Other Pacific Islander
# Non-Hispanic White
# Not Proficient in English
# Primary Care Physicians
# Rural
# Single-Parent Households
# Some College
# Unemployed
# Uninsured
# Uninsured
# Uninsured.1
# Workers who Drive Alone
% 65 and older CI high
% 65 and older CI low
% 65 and older denominator
% 65 and older numerator
% 65 and older raw value
% 65 and over
% < 18
% African American
% Alcohol-Impaired
% American Indian and Alaskan Native CI high
% American Indian and Alaskan Native CI low
% American Indian and Alaskan Native denominator
% American Indian and Al

In [8]:
# Manipulate column names
subs = [(' ', '_'),('.',''),("'",""),('™', ''), ('®',''),
        ('+','plus'), ('½','half'), ('-','_'), ('#','Num_'), 
        ('%', 'Per_'), ('<', 'Greater_than_'), ('>', 'Less_than_')
       ]

def col_formatting(col):
    new_cols = []
    for line in col:
        for sub in subs:
            line = line.replace(sub[0], sub[1])
        new_cols.append(line)
    return new_cols

In [9]:
col_new = col_formatting(combimed.columns)
combimed.columns = col_new

## Data Exploration:

Create dataframe for description of columns + Re import , drop first row 

In [10]:
def data_nUnique_values(df):
    print("DataFrame Shape:", df.shape)
    print("----------------------------")
    for n in df.columns:
        if df[n].isna().sum() != df.shape[0]:
            print(n, df[n].nunique(), df[n].isna().sum())

In [11]:
# data_nUnique_values(combimed)

## Set Target and Predictors:

In [12]:
outcomes = ['Life_Expectancy',
 'Life_Expectancy_(Black)',
 'Life_Expectancy_(Hispanic)',
 'Life_Expectancy_(White)',
 'Life_expectancy_(Black)',
 'Life_expectancy_(Hispanic)',
 'Life_expectancy_(White)',
 'Life_expectancy_CI_high',
 'Life_expectancy_CI_low',
 'Life_expectancy_denominator',
 'Life_expectancy_numerator',
 'Life_expectancy_raw_value']

predictors = list(combimed.columns)

In [13]:
predictors = [x for x in predictors if x not in outcomes]

In [14]:
df_y = combimed[['Life_Expectancy',
 'Life_Expectancy_(Black)',
 'Life_Expectancy_(Hispanic)',
 'Life_Expectancy_(White)',
 'Life_expectancy_(Black)',
 'Life_expectancy_(Hispanic)',
 'Life_expectancy_(White)',
 'Life_expectancy_CI_high',
 'Life_expectancy_CI_low',
 'Life_expectancy_denominator',
 'Life_expectancy_numerator',
 'Life_expectancy_raw_value']]

In [15]:
df_y.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3195 entries, 0 to 3194
Data columns (total 12 columns):
Life_Expectancy                3073 non-null float64
Life_Expectancy_(Black)        1339 non-null float64
Life_Expectancy_(Hispanic)     1247 non-null float64
Life_Expectancy_(White)        1726 non-null float64
Life_expectancy_(Black)        1340 non-null object
Life_expectancy_(Hispanic)     1248 non-null object
Life_expectancy_(White)        1727 non-null object
Life_expectancy_CI_high        3125 non-null object
Life_expectancy_CI_low         3125 non-null object
Life_expectancy_denominator    0 non-null object
Life_expectancy_numerator      0 non-null object
Life_expectancy_raw_value      3125 non-null object
dtypes: float64(4), object(8)
memory usage: 299.7+ KB


In [16]:
def data_type(df, cols, datatype='str'):
    for col in cols:
        df[col] = df[col].astype(datatype)
    
    return df.info()

In [17]:
data_type(df_y, outcomes, 'float')

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3195 entries, 0 to 3194
Data columns (total 12 columns):
Life_Expectancy                3073 non-null float64
Life_Expectancy_(Black)        1339 non-null float64
Life_Expectancy_(Hispanic)     1247 non-null float64
Life_Expectancy_(White)        1726 non-null float64
Life_expectancy_(Black)        1340 non-null float64
Life_expectancy_(Hispanic)     1248 non-null float64
Life_expectancy_(White)        1727 non-null float64
Life_expectancy_CI_high        3125 non-null float64
Life_expectancy_CI_low         3125 non-null float64
Life_expectancy_denominator    0 non-null float64
Life_expectancy_numerator      0 non-null float64
Life_expectancy_raw_value      3125 non-null float64
dtypes: float64(12)
memory usage: 299.7 KB


/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  This is separate from the ipykernel package so we can avoid doing imports until


In [18]:
life_mean = df_y.mean(axis=1)

Target = life_mean

predictors = "everything else"

# Create Model DataFrame

In [19]:
predictors

['State_FIPS_Code',
 'County_FIPS_Code',
 '5_digit_FIPS_Code',
 'State_Abbreviation',
 'Name',
 'Release_Year',
 'County_Ranked_(Yes=1/No=0)',
 'Premature_death_raw_value',
 'Premature_death_numerator',
 'Premature_death_denominator',
 'Premature_death_CI_low',
 'Premature_death_CI_high',
 'Premature_death_(Black)',
 'Premature_death_(Hispanic)',
 'Premature_death_(White)',
 'Poor_or_fair_health_raw_value',
 'Poor_or_fair_health_numerator',
 'Poor_or_fair_health_denominator',
 'Poor_or_fair_health_CI_low',
 'Poor_or_fair_health_CI_high',
 'Poor_physical_health_days_raw_value',
 'Poor_physical_health_days_numerator',
 'Poor_physical_health_days_denominator',
 'Poor_physical_health_days_CI_low',
 'Poor_physical_health_days_CI_high',
 'Poor_mental_health_days_raw_value',
 'Poor_mental_health_days_numerator',
 'Poor_mental_health_days_denominator',
 'Poor_mental_health_days_CI_low',
 'Poor_mental_health_days_CI_high',
 'Low_birthweight_raw_value',
 'Low_birthweight_numerator',
 'Low_birthw

In [20]:
combimed.columns

Index(['State_FIPS_Code', 'County_FIPS_Code', '5_digit_FIPS_Code',
       'State_Abbreviation', 'Name', 'Release_Year',
       'County_Ranked_(Yes=1/No=0)', 'Premature_death_raw_value',
       'Premature_death_numerator', 'Premature_death_denominator',
       ...
       '95Per__CI___High17', 'Quartile33', 'Per__Drive_Alone_(Black)',
       'Per__Drive_Alone_(Hispanic)', 'Per__Drive_Alone_(White)',
       'Num__Workers_who_Drive_Alone', 'Per__Long_Commute___Drives_Alone',
       '95Per__CI___Low18', '95Per__CI___High18', 'Quartile34'],
      dtype='object', length=807)

In [21]:
df_model = combimed[predictors]

In [22]:
df_model['Life_Expectancy_Mean'] = life_mean

/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [23]:
df_model.head()

,State_FIPS_Code,County_FIPS_Code,5_digit_FIPS_Code,State_Abbreviation,Name,Release_Year,County_Ranked_(Yes=1/No=0),Premature_death_raw_value,Premature_death_numerator,Premature_death_denominator,...,Quartile33,Per__Drive_Alone_(Black),Per__Drive_Alone_(Hispanic),Per__Drive_Alone_(White),Num__Workers_who_Drive_Alone,Per__Long_Commute___Drives_Alone,95Per__CI___Low18,95Per__CI___High18,Quartile34,Life_Expectancy_Mean
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,3,82.722037,NaN,83.710021,20911.0,38.3,34.362673,42.237327,3,75.756145
1,00,000,00000,US,United States,2019,NaN,6900.630354,3745538,908266078,...,2,84.921904,68.343876,83.799135,74415.0,40.5,38.183480,42.816520,3,81.142409
2,01,000,01000,AL,Alabama,2019,NaN,9917.2328984,80440,13636816,...,1,80.154701,NaN,86.245265,7242.0,33.8,28.867176,38.732824,2,75.637862
3,01,001,01001,AL,Autauga County,2019,1,8824.0571232,815,156132,...,3,NaN,NaN,NaN,6930.0,48.6,40.310796,56.889204,4,75.032760
4,01,003,01003,AL,Baldwin County,2019,1,7224.6321603,2827,576496,...,3,NaN,81.767181,83.414506,18426.0,59.7,55.480714,63.919286,4,79.475268


# Train Test Split

In [ ]:
# Check every column is a float?
data_type()

In [ ]:
train, test = train_test_split(df_model, )